# Test Refactored Data Prep - Using `get_ds_resampled_gee()` and Step-by-Step Workflow

This notebook demonstrates two approaches:

## Part 1: Using Existing Asset (Quick Start)
- Loads monthly composites from GEE asset
- Processes them (renames bands, adds indices, smooths)
- Converts to xarray and saves to zarr

## Part 2: Step-by-Step Workflow (Create from Scratch)
- Creates ImageCollection from satellite data
- Filters by cloud cover
- Creates monthly composites
- Adds spectral indices and smoothing
- Converts to xarray and saves to zarr
- Optional: Export to GEE asset for reuse


In [2]:
import ee, eemont
import pandas as pd
import numpy as np
from forestry_carbon_arr.core import ForestryCarbonARR

# Initialize Forestry Carbon ARR system
forestry = ForestryCarbonARR(config_path='./00_input/korindo.json')
print("✅ Forestry Carbon ARR initialized")


/opt/venv/lib/python3.12/site-packages/gcsfs/core.py:317: UserWarning: GCS project not set - cannot list or create buckets
  warnings.warn("GCS project not set - cannot list or create buckets")


✅ Forestry Carbon ARR initialized


---

## Part 1: Using Existing Asset (Quick Start)

**Asset:** `projects/remote-sensing-476412/assets/korindo_sentinel2_monthly`
- Contains monthly composites (with `_median` suffix on band names)
- Needs band renaming + spectral indices + smoothing


In [3]:
# Load monthly composites from GEE asset and process
ds_resampled = forestry.get_ds_resampled_gee(
    use_existing_asset=True,  # ← Load from asset
    asset_folder='projects/remote-sensing-476412/assets/korindo_sentinel2_monthly',
    asset_is_monthly_composites=True,  # ← Asset contains monthly composites (needs processing)
    save_to_zarr=False,  # Save to zarr after processing
    zarr_path=None,  # Auto-detects from GCS_ZARR_DIR env var
    overwrite_zarr=False  # Don't overwrite if exists
)

print("\n✅ WORKFLOW COMPLETE")
print(f"📊 Final dataset: {dict(ds_resampled.sizes)}")
print(f"   Variables: {list(ds_resampled.data_vars)}")
print(f"\n✅ ds_resampled ready for tsfresh processing")


/opt/venv/lib/python3.12/site-packages/geemap/conversion.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


✓ GEE Initialized successfully
  Credentials Path: /usr/src/app/user_id.json - loaded successfully


/usr/src/app/forestry_carbon_arr/utils/gee_processing.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid_lon = aoi_wgs84.geometry.centroid.x.mean()
/usr/src/app/forestry_carbon_arr/utils/gee_processing.py:48: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid_lat = aoi_wgs84.geometry.centroid.y.mean()
/usr/src/app/forestry_carbon_arr/core/main.py:1008: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  self.logger.info(f"Dataset created: {dict(ds.dims)}")
⚠️  Dataset is still lazy-loaded from GEE. Acc


✅ WORKFLOW COMPLETE
📊 Final dataset: {'time': 81, 'X': 4489, 'Y': 3213}
   Variables: ['NDVI', 'EVI']

✅ ds_resampled ready for tsfresh processing


In [4]:
ds_resampled

<xarray.Dataset> Size: 9GB
Dimensions:  (time: 81, X: 4489, Y: 3213)
Coordinates:
  * time     (time) datetime64[ns] 648B 2018-02-15 2018-05-15 ... 2025-08-15
  * X        (X) float64 36kB 5.786e+05 5.786e+05 ... 6.235e+05 6.235e+05
  * Y        (Y) float64 26kB 9.949e+06 9.949e+06 ... 9.982e+06 9.982e+06
Data variables:
    NDVI     (time, X, Y) float32 5GB ...
    EVI      (time, X, Y) float32 5GB ...
Attributes:
    crs:      EPSG:32749

In [6]:
# This will trigger actual computation and show it's processing
print(ds_resampled['NDVI'].isel(time=0, X=100, Y=100).values)
# Or check if NDVI and EVI exist (they should, from processing)
print(list(ds_resampled.data_vars))

nan
['NDVI', 'EVI']


---

## Part 2: Step-by-Step Workflow (Create from Scratch)

This section shows the complete workflow when creating ImageCollection from scratch:
1. Create ImageCollection and prepare (UTM + reproject)
2. Filter by cloud cover
3. Create monthly composites
4. Rename bands (remove `_median` suffix)
5. Add spectral indices (NDVI, EVI, etc.)
6. Apply smoothing (Savitzky-Golay filter)
7. Convert to xarray using xee
8. Save/load from zarr

**Note:** This workflow can also export the processed ImageCollection to GEE asset for later reuse (to avoid concurrent request errors).


In [4]:
# STEP 1: Load AOI and initialize
print("=" * 60)
print("STEP 1: Load AOI and initialize")
print("=" * 60)

from forestry_carbon_arr.core.utils import DataUtils
import geopandas as gpd
import geemap

data_utils = DataUtils(forestry.config, use_gee=True)
aoi_gpd, aoi_ee = data_utils.load_geodataframe_gee(forestry.config["AOI_path"])

print(f"✅ AOI loaded: {len(aoi_gpd)} features")
print(f"   Area: {aoi_gpd.geometry.area.sum()/10000:.2f} hectares")


STEP 1: Load AOI and initialize
✅ AOI loaded: 1 features
   Area: 0.00 hectares


/tmp/ipykernel_3957/957302362.py:14: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  print(f"   Area: {aoi_gpd.geometry.area.sum()/10000:.2f} hectares")


In [5]:
# STEP 2: Create ImageCollection and prepare (UTM + reproject)
print("\n" + "=" * 60)
print("STEP 2: Create ImageCollection and prepare (UTM)")
print("=" * 60)

from forestry_carbon_arr.utils.gee_processing import prepare_image_collection_for_processing

raw_collection, utm_crs, pixel_scale, utm_epsg = prepare_image_collection_for_processing(
    config=forestry.config,
    aoi_gpd=aoi_gpd,
    aoi_ee=aoi_ee,
    years_back=1,  # Use 1 year for quick test
    use_existing_asset=False,  # Create from scratch
    import_strategy=forestry.import_strategy,
    reproject_to_utm_flag=True
)

print(f"✅ ImageCollection created: {raw_collection.size().getInfo()} images")
print(f"   UTM CRS: {utm_crs}")
print(f"   Pixel scale: {pixel_scale}m")



STEP 2: Create ImageCollection and prepare (UTM)
selecting Sentinel images


/usr/src/app/forestry_carbon_arr/utils/gee_processing.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid_lon = aoi_wgs84.geometry.centroid.x.mean()
/usr/src/app/forestry_carbon_arr/utils/gee_processing.py:48: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid_lat = aoi_wgs84.geometry.centroid.y.mean()


✅ ImageCollection created: 182 images
   UTM CRS: EPSG:32749
   Pixel scale: 10m


In [6]:
# STEP 3: Filter by cloud cover
print("\n" + "=" * 60)
print("STEP 3: Filter by cloud cover")
print("=" * 60)

from forestry_carbon_arr.utils.gee_processing import filter_by_cloud_cover

# Convert AOI to UTM for bounds calculation
aoi_gpd_utm = aoi_gpd.to_crs(utm_crs)
aoi_ee_utm_geom = geemap.geopandas_to_ee(aoi_gpd_utm).geometry()
aoi_bounds_utm = aoi_ee_utm_geom.bounds(maxError=1)

valid_pixel_threshold = forestry.config.get('valid_pixel_threshold', 70.0)
collection_filtered, stats = filter_by_cloud_cover(
    raw_collection,
    aoi_bounds_utm,
    scale=pixel_scale,
    crs=utm_crs,
    valid_pixel_threshold=valid_pixel_threshold
)

print(f"✅ Filtered collection: {collection_filtered.size().getInfo()} images")
print(f"   Retention rate: {stats.get('retention_rate', 0):.1f}%")



STEP 3: Filter by cloud cover
✅ Filtered collection: 112 images
   Retention rate: 61.5%


In [ ]:
# STEP 4: Create monthly composites and rename bands
print("\n" + "=" * 60)
print("STEP 4: Create monthly composites and rename bands")
print("=" * 60)

from forestry_carbon_arr.utils.gee_processing import create_monthly_composites, rename_composite_bands

monthly_images = create_monthly_composites(
    collection_filtered,
    aoi_ee.geometry(),
    reducer='median'
)

# Create ImageCollection from monthly images
collection_monthly_raw = ee.ImageCollection(monthly_images).sort('system:time_start')
print(f"   Raw monthly collection: {collection_monthly_raw.size().getInfo()} images")

# Rename bands: remove '_median' suffix and exclude 'cloudM_median'
collection_monthly = rename_composite_bands(
    collection_monthly_raw,
    remove_suffix='_median',
    exclude_bands=['cloudM']
)

print(f"✅ Monthly ImageCollection: {collection_monthly.size().getInfo()} images")
band_names_monthly = collection_monthly.first().bandNames().getInfo()
print(f"   Bands: {', '.join(band_names_monthly[:5])}{'...' if len(band_names_monthly) > 5 else ''}")


In [ ]:
# STEP 5: Add spectral indices and apply smoothing
print("\n" + "=" * 60)
print("STEP 5: Add spectral indices and apply smoothing")
print("=" * 60)

from forestry_carbon_arr.utils.gee_processing import process_collection_with_indices_and_smoothing

collection_with_sg = process_collection_with_indices_and_smoothing(
    collection=collection_monthly,
    config=forestry.config,
    aoi_ee=aoi_ee,
    spectral_bands=['NDVI', 'EVI'],
    smoothing_window=3,  # Small window for 1 year test
    smoothing_polyorder=2,
    add_fcd=False
)

print(f"✅ Final collection: {collection_with_sg.size().getInfo()} images")
band_names_sg = collection_with_sg.first().bandNames().getInfo()
print(f"   Bands: {band_names_sg}")


### Optional: Export to GEE Asset (to avoid concurrent requests)

**Note:** This step is optional but recommended for large datasets. Exporting the processed ImageCollection to a GEE asset allows you to:
- Avoid "Too many concurrent aggregations" errors
- Reuse the processed collection later (like in Part 1)
- Share the collection with other users/projects

**To export, uncomment and run the cell below:**


In [ ]:
# OPTIONAL: Export processed ImageCollection to GEE asset
# Uncomment to export (this takes time, so skip for quick testing)

# asset_folder = 'projects/remote-sensing-476412/assets/korindo_sentinel2_monthly'
# 
# print(f"\n{'=' * 60}")
# print("EXPORT: Saving processed ImageCollection to GEE asset")
# print(f"{'=' * 60}")
# print(f"   Asset folder: {asset_folder}")
# print("   This may take several minutes...")
# 
# # Export each image in the collection
# collection_list = collection_with_sg.toList(collection_with_sg.size())
# n_images = collection_list.size().getInfo()
# 
# for i in range(n_images):
#     img = ee.Image(collection_list.get(i))
#     img_id = img.get('system:id').getInfo() or f"Sentinel2_{i:04d}"
#     asset_id = f"{asset_folder}/{img_id}"
#     
#     print(f"   Exporting image {i+1}/{n_images}: {asset_id}")
#     
#     task = ee.batch.Export.image.toAsset(
#         image=img,
#         description=f"Export {img_id}",
#         assetId=asset_id,
#         region=aoi_ee.geometry(),
#         scale=pixel_scale,
#         crs=utm_crs,
#         maxPixels=1e13
#     )
#     task.start()
# 
# print(f"\n✅ Export tasks started. Check GEE Tasks tab to monitor progress.")
# print("   Once complete, you can use this asset in Part 1 with:")
# print(f"   asset_folder='{asset_folder}'")


In [ ]:
# STEP 6: Convert to xarray using xee
print("\n" + "=" * 60)
print("STEP 6: Convert to xarray using xee")
print("=" * 60)

import xarray as xr

# Convert AOI geometry to UTM for xee
aoi_gpd_utm = aoi_gpd.to_crs(utm_crs)
aoi_gpd_utm.set_crs(utm_crs, inplace=True)
aoi_ee_utm_geom = geemap.geopandas_to_ee(aoi_gpd_utm).geometry()

print(f"   CRS: {utm_crs}")
print(f"   Scale: {pixel_scale}m")
print("   Converting...")

ds = xr.open_dataset(
    collection_with_sg,
    engine='ee',
    crs=utm_crs,
    scale=pixel_scale,
    geometry=aoi_ee_utm_geom
)

print(f"✅ Dataset created: {dict(ds.sizes)}")
print(f"   Data variables: {list(ds.data_vars)}")


In [ ]:
# STEP 7: Rename dimensions and save/load from zarr
print("\n" + "=" * 60)
print("STEP 7: Save to zarr and reload (to avoid GEE server errors)")
print("=" * 60)

from forestry_carbon_arr.utils.zarr_utils import save_dataset_efficient_zarr, load_dataset_zarr
import os

# Rename dimensions (X, Y -> x, y)
ds_resampled = ds.rename({'X': 'x', 'Y': 'y'})

# Determine zarr path
zarr_path = os.getenv('GCS_ZARR_DIR', '')
if zarr_path:
    if not zarr_path.startswith('gs://'):
        zarr_path = f"gs://{zarr_path}/ds_resampled_test.zarr"
    else:
        zarr_path = f"{zarr_path}/ds_resampled_test.zarr"
    storage = 'gcs'
    print(f"   Saving to GCS: {zarr_path}")
else:
    zarr_path = os.path.join(os.getcwd(), 'data', 'ds_resampled_test.zarr')
    storage = 'local'
    print(f"   Saving to local: {zarr_path}")

# Save to zarr
chunk_sizes = {'time': 40, 'x': 1024, 'y': 1024}
print("   Saving dataset to zarr...")
save_dataset_efficient_zarr(
    ds_resampled,
    zarr_path,
    chunk_sizes=chunk_sizes,
    compression='lz4',
    compression_level=1,
    overwrite=True,
    storage=storage
)
print(f"✅ Dataset saved to zarr")

# Load from zarr to avoid GEE server-side errors
print(f"\n   Loading from: {zarr_path}")
ds_resampled = load_dataset_zarr(zarr_path, storage=storage)
print(f"✅ Dataset loaded from zarr")
print(f"   Dataset: {dict(ds_resampled.sizes)}")
print(f"   Variables: {list(ds_resampled.data_vars)}")
print(f"\n✅ ds_resampled ready for tsfresh processing")


## Summary

This notebook demonstrates two approaches:

### Part 1: Using Existing Asset (Quick Start)
- Loads monthly composites from GEE asset
- Processes them (renames bands, adds indices, smooths)
- Converts to xarray and saves to zarr
- **Use this when:** You already have monthly composites exported to GEE

### Part 2: Step-by-Step Workflow (Create from Scratch)
- Creates ImageCollection from satellite data
- Filters by cloud cover
- Creates monthly composites
- Adds spectral indices and smoothing
- Converts to xarray and saves to zarr
- **Use this when:** You need to create everything from scratch
- **Optional:** Export processed collection to GEE asset for reuse

**Both approaches result in `ds_resampled` ready for tsfresh feature extraction!**
